In [1]:
# For cloning codes of Contrastive Learning
# It will be linked with finx_torch

In [36]:
import torch
import logging # logging을 안쓰는 model이 없다. 사용법 꼭 숙지!
# 간단히 말하면, 오류 발생시 출력해주는 놈!
import numpy as np
import torch.nn as nn
from utils import Metric # utils.py 구현해야함.
from scipy.stats import pearsonr,spearmanr
from sklearn.metrics.pairwise import paired_cosine_distances,paired_euclidean_distances,paired_manhattan_distances
# 코사인 거리, 유클리디안 거리, 맨해튼 거리 -> 이거 어디서 쓰는 거였지...?

In [32]:
logger =  logging.getLogger(__name__)
# 오류의 이름을 설정하고 
# 이후에 오류의 레벨을 결정한다.
# https://m.blog.naver.com/qvinci/221919758080

In [33]:
# similarity들의 계산의 결과를 살펴보자.

import torch.nn as nn

A = torch.randn([5,4,3])
B = torch.randn([7,4,3])
C = torch.randn([9,4,3])

positive_similarity = nn.CosineSimilarity(dim=-1)(A.unsqueeze(1), B.unsqueeze(0))
negative_similarity = nn.CosineSimilarity(dim=-1)(A.unsqueeze(1), C.unsqueeze(0))
cosine_similarity = torch.cat([positive_similarity, negative_similarity], dim=1)

positive_similarity.shape , negative_similarity.shape , cosine_similarity.shape

labels = torch.arange(cosine_similarity.size(0)).long()

cosine_similarity.shape ,cosine_similarity.size(0) , labels.shape

        

(torch.Size([5, 16, 4]), 5, torch.Size([5]))

In [41]:
class Loss():

    def __init__(self, args): #참고:args는 arguments를 뜻한다.
        self.args = args
        self.cos = nn.CosineSimilarity(dim=-1)
        # cos_sim 계산 시, 두 집단
        self.metric = Metric(args)

    def train_loss_fct(self,config,a,p,n):
        
        # 이 config는 자꾸 어디서 가져오는거지;;
        # 본격적인 contrastive learning
        
        positive_similarity = self.cos(a.unsqueeze(1), p.unsqueeze(0)) / self.args.temperature
        negative_similarity = self.cos(a.unsqueeze(1), n.unsqueeze(0)) / self.args.temperature
        
        # CosineSimilarity계산 시, 내적 계산이 들어가므로, 
        # 투입되는 두 요소가 하나는 row, 나머지 하나는 column
        #
        # 이 두 값은 이 후, 거리 계산 시에 자연 상수 e의 지수로 올라감.
        #  
        # a = h_i , p = h_i_+ , n = h_j
        # a와 p는 서로 pair인 단어들의 hidden_states
        # a와 n은 서로 pair가 아닌 단어들의 hidden_states
        # 
        # self.args.temperature => temperature hyper_parameter 
        # temperature param에 대한 설명 => https://velog.io/@jkl133/temperature-parameter-in-learner-fastai
        
        cosine_similarity = torch.cat([positive_similarity, negative_similarity], dim=1).to(self.args.device)
      
        # 위의 cosine_similarity는 아직 뭘 위해서 정의한건지는 모르겠음.
        # similarity들의 size는 a = (b1,r,c),p = (b2,r,c),n = (b3,r,c) 라 했을때
        # positive_similarity = (b1,b2,r)
        # negative_similarity = (b1,b3,r)
        # 두개 concat = (b1,b2+b3,r)
        # 단 r과 c는 셋 다 동일한 값을 가져야 한다.

        labels = torch.arange(cosine_similarity.size(0)).long().to(self.args.device)

        loss = config["criterion"](cosine_similarity,labels)
        
        return loss

In [42]:
def evaluation_during_training(self, embeddings1, embeddings2, labels, indicator):

        embeddings1 = embeddings1.cpu().numpy()
        embeddings2 = embeddings2.cpu().numpy()
        labels = labels.cpu().numpy().flatten()

        cosine_scores = 1 - (paired_cosine_distances(embeddings1, embeddings2))
        manhattan_distances = -paired_manhattan_distances(embeddings1, embeddings2)
        euclidean_distances = -paired_euclidean_distances(embeddings1, embeddings2)
        dot_products = [np.dot(emb1, emb2) for emb1, emb2 in zip(embeddings1, embeddings2)]

        eval_pearson_cosine, _ = pearsonr(labels, cosine_scores)
        eval_spearman_cosine, _ = spearmanr(labels, cosine_scores)

        eval_pearson_manhattan, _ = pearsonr(labels, manhattan_distances)
        eval_spearman_manhattan, _ = spearmanr(labels, manhattan_distances)

        eval_pearson_euclidean, _ = pearsonr(labels, euclidean_distances)
        eval_spearman_euclidean, _ = spearmanr(labels, euclidean_distances)

        eval_pearson_dot, _ = pearsonr(labels, dot_products)
        eval_spearman_dot, _ = spearmanr(labels, dot_products)

        score = {'eval_pearson_cosine': eval_pearson_cosine,
                 'eval_spearman_cosine': eval_spearman_cosine,
                 'eval_pearson_manhattan': eval_pearson_manhattan,
                 'eval_spearman_manhattan': eval_spearman_manhattan,
                 'eval_pearson_euclidean': eval_pearson_euclidean,
                 'eval_spearman_euclidean': eval_spearman_euclidean,
                 'eval_pearson_dot': eval_pearson_dot,
                 'eval_spearman_dot': eval_spearman_dot}

        self.metric.update_indicator(indicator, score)

        return max(eval_spearman_cosine, eval_spearman_manhattan, eval_spearman_euclidean, eval_spearman_dot)
        